1. Loading dependencies.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats





df = pd.read_csv("..\\data\\benin-malanville.csv")
#print(df.head(10))
#print(df.describe())



2. Summary Statistics & Missing-Value Report

In [7]:
print("Description of the numerical columns:")
print(df.describe())
print("Description of the categorical columns:")
print(df.describe(include=['object']))
print("Description of all columns:")
print(df.describe(include='all'))
print("An overview of the dataset:")
print(df.head())
print("The number of rows and columns in the dataset:")
print(df.shape)
print("An overview of the dataset:")
print(df.tail())
print("An overview of the dataset: 5 random rows")
print(df.sample(5))
print("An overview of the dataset: the shape of the dataset")
print(df.info())
print("Check for missing values:")
print(df.isna().sum())


Description of the numerical columns:
                 GHI            DNI            DHI           ModA  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      240.559452     167.187516     115.358961     236.589496   
std       331.131327     261.710501     158.691074     326.894859   
min       -12.900000      -7.800000     -12.600000       0.000000   
25%        -2.000000      -0.500000      -2.100000       0.000000   
50%         1.800000      -0.100000       1.600000       4.500000   
75%       483.400000     314.200000     216.300000     463.700000   
max      1413.000000     952.300000     759.200000    1342.300000   

                ModB           Tamb             RH             WS  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      228.883576      28.179683      54.487969       2.121113   
std       316.536515       5.924297      28.073069       1.603466   
min         0.000000      11.000000       2.100000       0.00000

3. Outlier Detection & Basic Cleaning

In [6]:
# Identify the numeric columns
# Compute Z-scores and flag outliers (|Z| > 3)
numeric_cols = []